## Langchain  - Chains

In [2]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate,PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [3]:
# Load environment variables from .env
load_dotenv()

True

In [4]:
# Create a ChatOpenAI model
chat = ChatOpenAI(model="gpt-4o")

LCEL :  LangChain Expression Language  is a simplified way of constructing LangChain chains.

* Write more concise and readable chains, and there is no need to import different modules.

* Single function to create various chain types, reducing boilerplate code.

* Built-in support for data processing steps and conditional logic within chains.

To chain in LCEL, we need to write the pipe bar and treat the pipe bar as a chain, so add a pipe bar between each variable

#### Example 1 # ## Chain

In [5]:
prompt = PromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [6]:
# Create the combined chain using LangChain Expression Language (LCEL)
chain = prompt | chat 
# chain = prompt_template | model

# Run the chain
result = chain.invoke({"product": "Queen Size Sheet Set"})
print(result.content)

Choosing the best name for a company that specializes in Queen Size Sheet Sets involves conveying the product's purpose, quality, and unique selling points. Here are a few suggestions:

1. **Queen's Comfort**
2. **Royal Rest Linens**
3. **Regal Sheets Co.**
4. **Crowning Bedding**
5. **Majestic Sleeps**
6. **Queenly Linens**
7. **Sovereign Sheets**
8. **Noble Dreams Bedding**
9. **Queen's Haven**
10. **Elegance Sheets**

Considerations for a name would include the target audience, brand values, and any particular style or theme the company wants to embody. A name that's easy to remember and suggests luxury or comfort can be very effective.


#### Output Parsers

#### Pydantic Output Parser

A Pydantic Output Parser for LLMs is a structured way to ensure that an LLM’s response is not free-form text, but validated, typed, and parsed into a predictable Python object using Pydantic models.

**⭐ Why do we need it?**

LLMs generate text. But in real systems we need reliable structured output:

- JSON with guaranteed fields

- Correct types (int, float, enums, lists, nested objects, etc.)

- Validation of constraints (e.g., score must be 0–1)

  

Pydantic solves this by defining a schema. The model is used to:

- Guide the prompt → tell the LLM the structure to follow

- Parse/validate the LLM’s output reliably

- Raise errors if output is incorrect → allowing retry or correction

#### 🧩 Example

**Step 1 — Define the Pydantic model**

In [5]:
from pydantic import BaseModel
from typing import List

class ProductInfo(BaseModel):
    name: str
    price: float
    tags: List[str]


#### Step 2 — Build an output parser (LangChain style

In [6]:
from langchain_core.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=ProductInfo)


#### Step 3 — Use parser instructions in the prompt

In [7]:
prompt = f"""
Extract product info from the text below.

{parser.get_format_instructions()}

Product description:
"Organic Honey costs 350 rupees and includes tags sweet, organic, grocery."
"""


In [8]:
prompt

'\nExtract product info from the text below.\n\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"title": "Name", "type": "string"}, "price": {"title": "Price", "type": "number"}, "tags": {"items": {"type": "string"}, "title": "Tags", "type": "array"}}, "required": ["name", "price", "tags"]}\n```\n\nProduct description:\n"Organic Honey costs 350 rupees and includes tags sweet, organic, grocery."\n'

#### Step 4 — Run the LLM and parse the output

In [19]:

raw_output = chat.invoke(prompt)

raw_output

AIMessage(content='```json\n{\n  "name": "Organic Honey",\n  "price": 350,\n  "tags": ["sweet", "organic", "grocery"]\n}\n```', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 226, 'total_tokens': 261, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_672b6a21ba', 'id': 'chatcmpl-CheyKayYBpplanMFktUxXaEHJ36e5', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--5085ea3c-9ba2-4f14-941e-141587d683db-0', usage_metadata={'input_tokens': 226, 'output_tokens': 35, 'total_tokens': 261, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [21]:
product = parser.parse(raw_output.content)
print(product)

name='Organic Honey' price=350.0 tags=['sweet', 'organic', 'grocery']


#### 🧩 Example 2

In [7]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

In [8]:
## Step 1 — Define the Pydantic model
# Let's start with defining how we would like the LLM output to look like:
# ### Parse the LLM output string into a Python dictionary

class Gift_Item(BaseModel):
    gift: bool = Field(description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
    delivery_days: int = Field(description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
    price_value: list = Field(description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

## Step 2 — Build an output parser (LangChain style)
# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Gift_Item)


customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

### Step 3 — Use parser instructions in the prompt

prompt = PromptTemplate(
    template=review_template_2,
    input_variables=["text"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

## Step 4 — Run the LLM and parse the output
# And a query intended to prompt a language model to populate the data structure.
prompt_and_model = prompt | chat
output = prompt_and_model.invoke({"text": customer_review})
parser.invoke(output)

Gift_Item(gift=True, delivery_days=2, price_value=["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."])